In [560]:
import csv
import time
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import requests
from bs4 import BeautifulSoup
import urllib
import os
import cv2
import json
import random

In [562]:
html_str = "<div id='msg' class='body strikeout'>Hello World!</div>"
soup = BeautifulSoup(html_str, "lxml")
tag = soup.div
print(tag.string)

Hello World!


In [619]:
#=============================================
#================ Main =======================
#=============================================
cat_page_path="https://www.cnyes.com/twstock/stock_astock.aspx"
category= get_cat(cat_page_path) #取得各類股路徑

cat_stock_id=[] #各分類的股票代碼 stock id
tmp_code_list=[]
cat_list=[] #各類股名稱 list

count=1
for i in category:
    tmp_code_list=get_cat_code(category[i]) #取得單一類股的股票代碼
    #tmp_code_list=['1402', '1409', '1410', '1413', '1414', '1416', '1417', \
    #               '1418', '1419', '1423', '1434', '1439', '1440', '1441', \
    #               '1443', '1444', '1445', '1446', '1447', '1449', '1451', \
    #               '1452', '1453', '1454', '1455', '1456', '1457', '1459', \
    #               '1460', '1463', '1464', '1465', '1466', '1467', '1468', \
    #               '1470', '1472', '1473', '1474', '1475', '1476', '1477', \
    #               '4414', '4426', '4438', '4439'] #測試用
    #print(tmp_code_list)
    
    cat_name=[] #各股名稱
    cat_NVP=[]  #各股淨值
    cat_PER=[]  #本益比
    cat_PBR=[]  #股價淨值比
    cat_ROE=[]  #股東權益報酬率
    cat_ROA=[]  #資產報酬率
    
    for j in tmp_code_list:
        a,b,c,d,e,f= single_stock2(j) #取得單一股票代碼的相關資料
        cat_name.append(a)
        cat_NVP.append(b)
        cat_PER.append(c)
        cat_PBR.append(d)
        cat_ROE.append(e)
        cat_ROA.append(f)
        count+=1
    #print(cat_NVP)
    print(" 類別= ", i,"\n"
        " NVP top3= ", get_top3(cat_NVP, cat_name, True),"\n", #取前三名
         "PER low3= ", get_top3(cat_PER, cat_name, reverse1=False),"\n", #取後三名
         "PBR top3= ", get_top3(cat_PBR, cat_name),"\n",
         "ROE top3= ", get_top3(cat_ROE, cat_name),"\n",
         "ROA top3= ", get_top3(cat_ROA, cat_name),"\n",
         "count= ", count)
    #print(tmp_list)
    cat_stock_id.append(tmp_code_list) #取得之單一類股的股票代碼放入 cat_stock_id
    cat_list.append(i) #類股名稱放入 cat_list


4
4
4
4
4
4
4
4
4
4
4
4
4
4
 類別=  水泥 
 NVP top3=  ['亞泥', '嘉泥', '台泥'] 
 PER low3=  ['東泥', '信大', '台泥'] 
 PBR top3=  ['台泥', '東泥', '信大'] 
 ROE top3=  ['台泥', '信大', '亞泥'] 
 ROA top3=  ['信大', '台泥', '亞泥'] 
 count=  8
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
4
4
4
 類別=  食品 
 NVP top3=  ['鮮活果汁-KY', '黑松', '興泰'] 
 PER low3=  ['大飲', '卜蜂', '南僑'] 
 PBR top3=  ['大統益', '鮮活果汁-KY', '佳格'] 
 ROE top3=  ['鮮活果汁-KY', '大統益', '佳格'] 
 ROA top3=  ['鮮活果汁-KY', '大統益', '佳格'] 
 count=  30
5
4
4
5
5
4
4
4
4
4
4
5
4
4
5
5
4
5
5
4
4
5
4
4
5
5
4
4
4
4
4
4
5
4
4
5
5
4
5
5
4
4
 類別=  塑膠 
 NVP top3=  ['達新', '台化', '台塑'] 
 PER low3=  ['炎洲', '炎洲', '炎洲'] 
 PBR top3=  ['恆大', '富林-KY', '台達化'] 
 ROE top3=  ['達新', '恆大', '台達化'] 
 ROA top3=  ['達新', '恆大', '台達化'] 
 count=  51
4
5
4
5
4
5
5
6
4
5
4
5
5
5
5
4
5
5
5
5
5
5
4
5
5
5
4
4
5
5
5
5
5
5
5
5
6
5
4
5
4
4
5
5
4
4
4
5
4
5
4
5
5
5
4
5
4
5
5
5
5
4
5
5
5
5
5
5
4
5
5
4
4
4
5
5
5
5
5
5
4
5
5
5
4
5
4
4
5
5
4
4
 類別=  紡織 
 NVP top3=  ['冠星-KY', '廣越', '儒鴻'

KeyboardInterrupt: 

In [258]:
#獲得各類股的股票代碼
def get_cat_code(category_path):
#name='水泥' #測試用
#path='https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=01&stitle=%e6%b0%b4%e6%b3%a5' #測試用
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                       "AppleWebKit/537.36 (KHTML, like Gecko)"
                       "Chrome/63.0.3239.132 Safari/537.36"}   #模擬一般瀏覽器
    r= requests.get(category_path,headers=headers)  #由提供之路徑獲得網頁資訊
    r.encoding='utf8'  #utf8編碼
    soup3= BeautifulSoup(r.text,"lxml")  #定義 beautiful soup並取得資料, 以lxml型式

    cat_code= soup3.find_all("div",attrs={"class":"TableBox"} ) #找尋所有 class 名稱為 TableBox 的資料

    cat_stock_id=[] #定義空 list
    count=1
    for i in cat_code:
        cat_code2= i.find_all("td") #取得 TableBox 下的 td 標籤資料

    for j in cat_code2:
        if count%11 ==2:         #取得第2個 td 標籤資料
            #print(j.string)
            cat_stock_id.append(j.string) #放入股票代碼回饋區
        count+=1
    return cat_stock_id  #傳回股票代碼

In [580]:
category_path="https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=01&stitle=%e6%b0%b4%e6%b3%a5"
cat_code_list=get_cat_code(category_path)
print(cat_code_list)

['1101', '1102', '1103', '1104', '1108', '1109', '1110']


In [579]:
cat_page_path="https://www.cnyes.com/twstock/stock_astock.aspx"
get_cat(cat_page_path)

{'水泥': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=01&stitle=%e6%b0%b4%e6%b3%a5',
 '食品': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=02&stitle=%e9%a3%9f%e5%93%81',
 '塑膠': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=03&stitle=%e5%a1%91%e8%86%a0',
 '紡織': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=04&stitle=%e7%b4%a1%e7%b9%94',
 '電機': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=05&stitle=%e9%9b%bb%e6%a9%9f',
 '電器': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=06&stitle=%e9%9b%bb%e5%99%a8',
 '玻璃': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=08&stitle=%e7%8e%bb%e7%92%83',
 '造紙': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=09&stitle=%e9%80%a0%e7%b4%99',
 '鋼鐵': 'https://www.cnyes.com/twstock/index2real.aspx?stockType=T&groupId=10&stitle=%e9%8b%bc%e9%90%b5',
 '橡膠': 'https://www.cnyes.com/twstock/index2real.aspx?s

In [565]:
#return stock_category & link
def get_cat(cat_page_path):
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                   "AppleWebKit/537.36 (KHTML, like Gecko)"
                   "Chrome/63.0.3239.132 Safari/537.36"} #模擬一般瀏覽器
    #cat_page_path="https://www.cnyes.com/twstock/stock_astock.aspx" #測試用
    clean= cat_page_path[:cat_page_path.rfind('/')+1] #取得前半部的路徑名稱
    #print(clean)
    r= requests.get(cat_page_path,headers=headers)
    r.encoding='utf8'
    soup2= BeautifulSoup(r.text,"lxml")
    #tag_cat= soup2.find_all("ul",attrs={"class":"kdlist"})

    tag_cat1= soup2.find(class_="disyshow",id="kinditem_0") #取得 class 名稱為 disyshow, id 名稱為 kinditem_0 的標籤資料
    tag_cat2= tag_cat1.find("ul") #取得上述資料之單一 ul 標籤資料
    tag_cat3= tag_cat2.find_all("li") #取得 ul 資料下的所有 li 標籤資料
    tag_cat4= tag_cat2.find_all("a") #取得 li 資料下的所有 a 標籤資料
    tag_link=[]
    tag_name=[]
    for j in tag_cat4:
        tag_link.append(clean+j.get('href')) #獲得類股資料連結
    for i in tag_cat3:
        tag_name.append(i.string) #取得類股名稱
    tag_cat=dict(zip(tag_name, tag_link)) #將類股名稱與類股細項路徑連結 (dict格式)
    #print(tag_cat) #測試用
    return tag_cat #回傳類股名稱與類股細項路徑

In [403]:
# Money DJ
# return stock name, NVP, PER, PBR, ROE, ROA
def single_stock(stock_id):
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                   "AppleWebKit/537.36 (KHTML, like Gecko)"
                   "Chrome/63.0.3239.132 Safari/537.36"}
    print("https://www.moneydj.com/z/zc/zca/zca_"+stock_id+".djhtm")
    r= requests.get("https://www.moneydj.com/z/zc/zca/zca_"+stock_id+".djhtm",headers=headers)
    r.encoding='big5'
    soup= BeautifulSoup(r.text,"lxml")

    #股票名稱
    tag_name= soup.find("td",attrs={"class":"t10"})
    stock=(tag_name.text.strip()).replace(' ','')[:10]
    stock_name= stock[:stock.find('(')]
    #print('股票名稱: ', stock_name)
    stock_code= ((stock[stock.find('('):]).replace('(','')).replace(')','').replace(' ','')
    #print('股票代碼: ', stock_code)
    #每股淨值
    tag_nvp= soup.find_all("p",attrs={"class":"t3n1"})
    #for i in tag_nvp:
    #print('每股淨值 NVP: ',tag_nvp[1].string)
    #本益比
    tag_per= soup.find_all("td",attrs={"class":"t3n1"})
    #print('本益比 PER: ',tag_per[7].string)
    #股價淨值比
    tag_pbr= soup.find_all("td",attrs={"class":"t3n1"})
    #print('股價淨值比 PBR: ', tag_pbr[34].string)
    #股東權益報酬率 ROE
    #print('股東權益報酬率 ROE: ', tag_pbr[35].string)
    #資產報酬率 ROA
    #print('資產報酬率 ROA: ', tag_pbr[32].string)
    # return stock name, NVP, PER, PBR, ROE, ROA
    return stock_name,tag_nvp[1].string,tag_per[7].string,tag_pbr[34].string, tag_pbr[35].string,tag_pbr[32].string

In [590]:
single_stock2('2330')

台積電


('台積電', '66.35', '25.31', '6.69', '7.12%', '5.06%')

In [591]:
# Yahoo stock
# return stock name, NVP, PER, PBR, ROE, ROA
# stock_id= 股票代碼; count1 及 count2 為系統休息時間, 預設為 1 秒
def single_stock2(stock_id,count1=1,count2=1):
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                   "AppleWebKit/537.36 (KHTML, like Gecko)"
                   "Chrome/63.0.3239.132 Safari/537.36"}
    #print("https://tw.screener.finance.yahoo.net/screener/ws?f=j&ShowID="+stock_id+"&574")
    r= requests.get("https://tw.stock.yahoo.com/d/s/company_"+stock_id+".html",headers=headers) #取得 name, NVP, ROE, ROA  的網頁路徑
    #https://tw.screener.finance.yahoo.net/screener/screen02.html?cps0=1101&cps1=1101%20%E5%8F%B0%E6%B3%A5%20&cps2=5471%20%E6%9D%BE%E7%BF%B0&cps3=2363%20%E7%9F%BD%E7%B5%B1&cps4=3189%20%E6%99%AF%E7%A2%A9&cps5=2337%20%E6%97%BA%E5%AE%8F
    #r2= requests.get("https://tw.screener.finance.yahoo.net/screener/screen02.html?cps0="+stock_id+"&cps1=1101%20%E5%8F%B0%E6%B3%A5%20&cps2=5471%20%E6%9D%BE%E7%BF%B0&cps3=2363%20%E7%9F%BD%E7%B5%B1&cps4=3189%20%E6%99%AF%E7%A2%A9&cps5=2337%20%E6%97%BA%E5%AE%8F",headers=headers)    
    r2= requests.get("https://tw.screener.finance.yahoo.net/screener/ws?f=j&ShowID="+stock_id+"&574",headers=headers) #取得 PER 及 PBR 的 JSON 路徑
    r.encoding='big5' #路徑1 用 big5 編碼解析
    r2.encoding='utf8' #路徑2 用 utf8 解析
    soup= BeautifulSoup(r.text,"lxml")
    stock_code=''
    stock_name=''

    #取得股票名稱
    tag_name= soup.find("tr",attrs={"class":"yui-stock-genpage-pright"})
    tag_name= tag_name.td.font.b.string
    for i in tag_name:
        if i.isdigit():
            stock_code+=str(i)
        else:
            stock_name+=i
    #print('股票名稱: ', stock_name) #測試用
    #print('股票代碼: ', stock_code) #測試用
    
    #取得每股淨值 nvp
    tag_nvp1= soup.find("td",attrs={"height":"25"},text='股東權益報酬率')
    tag_nvp=tag_nvp1.next_sibling.next_sibling.next_sibling.next_sibling.string #取得 tag_nvp1 的旁邊的旁邊標籤內容
    tag_nvp= tag_nvp[(tag_nvp.find(': 　　')+1):tag_nvp.find('元')].strip() #將取得內容清理獲得乾淨資料, 包含取得數值及清除空白
    #print('每股淨值 NVP: ',tag_nvp) #測試用
    
    #系統休息
    if (count1%10==0):
        time.sleep(random.randint(3,5))
    if (count2%50==0):
        time.sleep(random.randint(10,20))
    
    #取得 PER 及 PBR
    while r2.status_code != requests.codes.ok: #若是沒取得資料，則一直重複下載
        r2= requests.get("https://tw.screener.finance.yahoo.net/screener/ws?f=j&ShowID="+stock_id+"&574",headers=headers) #取得json
        r2.encoding='utf8' #用utf8編碼
        
    j=json.loads(r2.text ) #取得json資料，資料型態為 dict
    #print(j)
    tag_per1=j.get("items") #取得名為 items 的對照資料 
    for k in tag_per1:
        tag_per= k.get("vFLD_PER") #取得 PER
        tag_pbr= k.get("vFLD_PBR") #取得 PBR
    if tag_per != "": #若不是空資料
        tag_per=tag_per[:tag_per.find('倍')] #則清理為乾淨資料  
    #print('本益比 PER: ',tag_per)
    #print('股價淨值比 PBR: ', tag_pbr)
    
    #取得 ROE
    tag_roe= tag_nvp1.next_sibling.next_sibling.string #取得剛剛每股淨值的旁邊標籤資料
    #print('股東權益報酬率 ROE: ',tag_roe)
    
    #取得 ROA
    tag_roa1=soup.find("td",attrs={"height":"25"},text='資產報酬率') #尋找td標籤，其性質為 高度25, 內容為 資產報酬率
    tag_roa=tag_roa1.next_sibling.next_sibling.string #取得找到資料的旁邊標籤資料
    #print('資產報酬率 ROA: ', tag_roa)
    
   # return stock name, NVP, PER, PBR, ROE, ROA
    #print(stock_name)
    return stock_name,tag_nvp,tag_per,tag_pbr, tag_roe,tag_roa

In [624]:
#獲得前三名資料
def get_top3(list_data, name, reverse1=True):
    #a= [1,2,3,4,5] 測試用
    tmp_list=[]
    tmp=0
    #資料清洗 
    for i in list_data:
        if i != '':
            if i.find('%')>0: #如果有 %
                tmp= (float)(i[:i.find('%')])/100 #去除 % 後，除100再轉成 float
            else:
                tmp=(float)(i.strip()) #轉成 float
        else:
            if (reverse1 == True):
                tmp=0.00 #空資料設為 0.00
            else:
                tmp=1000000.00
        tmp_list.append(tmp)
    list_data=tmp_list #蒐集為 list
    
    #print(list_data)
    name_dic=dict(zip(list_data, name)) #將'資料'與'股票名稱'連結成 dict
    (list_data).sort(reverse=reverse1) #將資料順項排序 (由大到小)
    count=0
    top3=[]
    for i in list_data: #取前三項
        if count <3:
            #print(i)
            top3.append(name_dic[i]) #由'資料'對照'股票名稱'
            count+=1
    #print(top3)
    return top3 #回傳前三股票名稱

In [623]:
check_ok('6452')

False

In [620]:
def check_ok(stock_id):
    ok_flag=True
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                   "AppleWebKit/537.36 (KHTML, like Gecko)"
                   "Chrome/63.0.3239.132 Safari/537.36"}
    r2= requests.get("https://tw.screener.finance.yahoo.net/screener/ws?f=j&ShowID="+stock_id+"&574",headers=headers) #取得 PER 及 PBR 的 JSON 路徑
    r2.encoding='utf8' #路徑2 用 utf8 解析
    while r2.status_code != requests.codes.ok: #若是沒取得資料，則一直重複下載
        r2= requests.get("https://tw.screener.finance.yahoo.net/screener/ws?f=j&ShowID="+stock_id+"&574",headers=headers) #取得json
        r2.encoding='utf8' #用utf8編碼       
    j=json.loads(r2.text ) #取得json資料，資料型態為 dict
    #print(j)
    tag_kValue=j.get("items") #取得名為 items 的對照資料 
    for k in tag_kValue:
        tag_ok= k.get("vFLD_K9_UPDNRATE") #取得 PER
    if (tag_ok == '--'):
        ok_flag= False
    return ok_flag